In [2]:
import sys
from aerospike import Client
from aerospike import predicates as pred

from set_proxy import SetProxy
from open_ai_gateway import OpenAIGateway
from pprint import pprint

In [3]:
def clean_and_aggregate(accum_list, callback_record):
    record_body = callback_record[2]
    record_body["itemId"] = int(record_body["itemId"])
    accum_list.append(record_body)

In [4]:
cluster_addrs = {
    "hosts": [("localhost", 3000)]
}
client = Client(cluster_addrs).connect( )

In [5]:
print(f"Connected to Aerospike server: = {client.get_node_names( )}")

Connected to Aerospike server: = [{'address': '::1', 'port': 3000, 'node_name': 'BB9A25B5F7ADD4A'}]


In [6]:
proxy = SetProxy(client, "inventory", "catalog")

In [7]:
cleaned_query_results = [ ]

In [8]:
query = proxy.query( )

In [9]:
# Simulate the scenario where a human web user is looking at a
# dress. Set a WHERE clause on the query object to prune the
# catalog only to dresses.
query.where(pred.equals("category", "dress"))

In [11]:
# Perform a projection operation (equivalent to a SELECT clause)
# to specify that we only want the feature values that we're
# interested in for making recommendations to be returned instead
# of the entire record and all of its metadata. This reduces
# network payload size. Note that we haven't included the "price"
# bin in our desired feature set -- for our simulated human web
# user, money is no object. :-)
query.select("itemId", "subcategory", "decade", "pattern", "color",
             "size", "material")

In [16]:
# Execute a foreach operation on the query. This will iterate
# through all of the records in the result set in a streaming
# fashion. We earlier defined a clean_and_aggregate method
# to remove extraneous record metadata and put all of our
# query results into a list. So let's use it now to put
# our query results into the cleaned_query_results list.
query.foreach(lambda curr_record: clean_and_aggregate(
    cleaned_query_results, curr_record))

In [17]:
# Open an OpenAI API gateway and examine its SYSTEM role.
# this role is a prompt that controls LLM reasoning
# behvior (as opposed to the USER role, which is just
# what you and I type into ChatGPT). SYSTEM role prompt
# engineering is an art form best left to very AI-literate
# data scientists.
gateway = OpenAIGateway( )
print(f"OpenAI Gateway System Role = {gateway.get_system_role( )}")

OpenAI Gateway System Role = You are a vintage fashion recommender for women's dresses.
        Task: pick the THREE closest matches in 'inventory' to the 'exemplar'.
        Judging criteria (in order):
        1) Silhouette/subcategory affinity (minidress ~ bodycon/shift > mumu/trapeze).
        2) Era proximity (1980s closest to late-70s/early-90s).
        3) Pattern match (solid vs plaid/striped/floral).
        4) Color family (exact or near jewel-tone match gets a bonus).
        5) Fabric/handfeel (smooth/stretchy cotton-blend ~ satin/poly jerseys > wool/flannel).
        6) Size tolerance: vintage can be tailored; +/-1–2 numeric or adjacent tee sizes are acceptable.
        Return STRICT JSON only with keys:
        - top3: array of 3 objects [{itemId, score (0..1), why}]
        - notes: one short paragraph explaining the trade-offs.
        


In [18]:
# Ask OpenAI to recommend other dresses similar to an
# exemplar dress, return result JSON, and explain its
# reasoning and confidence in its recommendations.
response = gateway.recommend(2767, cleaned_query_results)

Making RAR (retrieval-augmented recommendation) call to OpenAI...
OpenAPI call complete. Round trip time = 5.842 sec.


In [19]:
# Print the results from OpenAI
pprint(response)

{'notes': "The top matches prioritize the 'formal' subcategory and 1970s era, "
          'which are the most critical factors. While none of the top matches '
          'have a striped pattern, they offer solid or houndstooth patterns '
          'with jewel-tone or near jewel-tone colors, which are the next best '
          'options. The fabric and size considerations were less critical due '
          'to the strong alignment in silhouette and era.',
 'top3': [{'itemId': 5182,
           'score': 0.8,
           'why': "Matches the 'formal' subcategory and 1970s era, but is "
                  'solid in pattern and mauve in color, which is a near '
                  'jewel-tone match.'},
          {'itemId': 2396,
           'score': 0.7,
           'why': "Matches the 'formal' subcategory and 1970s era, with a "
                  'houndstooth pattern and garnet color, which is a jewel-tone '
                  'match.'},
          {'itemId': 3856,
           'score': 0.6,
          